In [3]:
!pip install tensorflow opencv-python




   ---------------------------------------- 0.0/385.0 MB ? eta -:--:--
   ---------------------------------------- 2.9/385.0 MB 13.9 MB/s eta 0:00:28
    --------------------------------------- 7.9/385.0 MB 18.7 MB/s eta 0:00:21
   - -------------------------------------- 12.1/385.0 MB 18.9 MB/s eta 0:00:20
   - -------------------------------------- 16.0/385.0 MB 18.6 MB/s eta 0:00:20
   -- ------------------------------------- 20.2/385.0 MB 18.7 MB/s eta 0:00:20
   -- ------------------------------------- 24.4/385.0 MB 18.8 MB/s eta 0:00:20
   -- ------------------------------------- 27.3/385.0 MB 18.2 MB/s eta 0:00:20
   --- ------------------------------------ 30.7/385.0 MB 17.7 MB/s eta 0:00:21
   --- ------------------------------------ 34.6/385.0 MB 17.7 MB/s eta 0:00:20
   --- ------------------------------------ 38.3/385.0 MB 17.8 MB/s eta 0:00:20
   ---- ----------------------------------- 41.9/385.0 MB 17.7 MB/s eta 0:00:20
   ---- ----------------------------------- 44.6/38

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import numpy as np
import cv2


In [ ]:

# Set the image dimensions and batch size
IMG_HEIGHT, IMG_WIDTH = 128, 128

# Load bounding box data from CSV
annotations_path = 'annotations.csv'
annotations = pd.read_csv(annotations_path)

# Initialize lists to store images and their corresponding bounding boxes
images = []
bounding_boxes = []

# Load images and corresponding bounding boxes
for index, row in annotations.iterrows():
    image_path = os.path.join('images', row['filename'])  # Assume all images are in a folder named 'images'
    image = cv2.imread(image_path)
    if image is not None:
        # Resize image to target size
        image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
        images.append(image)
        
        # Extract bounding box coordinates
        bbox = row[['x', 'y', 'width', 'height']].values
        bounding_boxes.append(bbox)

# Convert lists to numpy arrays
X = np.array(images, dtype='float32')
y = np.array(bounding_boxes, dtype='float32')

# Normalize the input images
X /= 255.0

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
def create_model():
    model = Sequential()
    
    # First Convolutional Layer
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Second Convolutional Layer
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Third Convolutional Layer
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten the layers
    model.add(Flatten())

    # Fully Connected Layer
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))

    # Output layer: two neurons for the coordinates of the bounding box (x, y, width, height)
    model.add(Dense(4, activation='sigmoid'))
    
    return model

model = create_model()
model.summary()


c:\Users\kumar\OneDrive\Documents\GenAI Projects\QnA_chatbot\myenv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 691,716 (2.64 MB)

 Trainable params: 691,716 (2.64 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
model.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
# Assume X_train is the training data and y_train are the bounding boxes
# X_train.shape -> (num_samples, height, width, channels)
# y_train.shape -> (num_samples, 4) for bounding box coordinates

# Normalize the input images
X_train = X_train / 255.0

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


In [ ]:
def predict_and_draw_boxes(image, model):
    # Preprocess the input image
    input_image = cv2.resize(image, (64, 64))
    input_image = input_image.astype('float32') / 255.0
    input_image = np.expand_dims(input_image, axis=0)

    # Predict the bounding box
    bbox = model.predict(input_image)[0]

    # Denormalize the bounding box coordinates
    h, w, _ = image.shape
    startX, startY, width, height = bbox
    startX = int(startX * w)
    startY = int(startY * h)
    endX = int(startX + width * w)
    endY = int(startY + height * h)

    # Draw the bounding box on the image
    cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)

    return image

# Test the model with a new image
image = cv2.imread('test_image.jpg')
output_image = predict_and_draw_boxes(image, model)

# Display the result
cv2.imshow("Detected Objects", output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

